In [1]:
import pandas as pd
import numpy as np

In [2]:
emails = pd.read_csv('DataSets/index/train.csv',sep=" ",header=None,names=['category','paths'])
emails['body'] = ''

category
ham     25220
spam    50199
dtype: int64

In [4]:
import email
import codecs
import re
from bs4 import BeautifulSoup
for index in range(0,len(emails.paths)):
    fp = codecs.open("DataSets"+emails.paths[index][2:], "r",encoding='utf-8', errors='ignore')
    email_text = email.message_from_file(fp)
    texts = ""
    if  email_text.is_multipart():
        for part in email_text.get_payload():
            if part.get_content_maintype() == 'text':
                texts += part.get_payload()
    else:
        texts += email_text.get_payload()
    texts = re.sub('\n','',texts)
    texts = re.sub('_','',texts)
    texts = re.sub('-',' ',texts)
    texts = re.sub('/',' ',texts)
    texts = re.sub(':',' ',texts)
    texts = re.sub('$','',texts)
    texts = re.sub('=','',texts)
    texts = re.sub('<.*?>', '', texts)
    emails.body[index] = " ".join((BeautifulSoup(texts.lower(), 'html.parser').findAll(text=True)))
    fp.close()
emails.groupby("category").size()

category
ham     25220
spam    50199
dtype: int64

In [5]:
temails = emails[emails.body!='']
temails=temails[['category','body']]
te_spam=temails.loc[temails['category']=='spam'].sample(50000,replace=True)
te_ham=temails.loc[temails['category']=='ham'].sample(50000,replace=True)
temails=pd.concat([te_spam,te_ham])
temails.groupby('category').size()

category
ham     50000
spam    50000
dtype: int64

In [7]:
import nltk
#nltk.download()
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        stemmer = SnowballStemmer("english", ignore_stopwords=True)
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

In [8]:
#applying bag of words algorithm
#segment each text file into words (for English splitting by space), and 
#count # of times each word occurs in each document and finally assign each word an integer id.
count_vect = StemmedCountVectorizer(stop_words='english').fit(temails.body)
X_train_counts = count_vect.transform(temails.body)
X_train_counts.shape

(100000, 418641)

In [9]:
# TF-IDF i.e Term Frequency times inverse document frequency.
# TF:  It will give more weightage to longer documents than shorter documents.
# IDF: Finally, we can even reduce the weightage of more common words like (the, is, an etc.) 
#      which occurs in all document.
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_train_tfidf.shape

(100000, 418641)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, temails.category )
clf.score(X_train_tfidf, temails.category)

In [ ]:
text = input("Enter Email body").lower()
X_predict = count_vect.transform(np.array([text]))
X_predict_tfidf = tfidf_transformer.transform(X_predict)
X_predict_tfidf.shape

In [ ]:
clf.predict(X_predict_tfidf)